In [ ]:
using Logging
logger = ConsoleLogger(stdout)
# debuglogger = ConsoleLogger(stderr, Logging.Debug)
global_logger(logger)
using JLD

using Random
rng = MersenneTwister(1234)
import Dates

In [ ]:
using Plots
pyplot()

In [ ]:
include("../utils/payoff_functions.jl")
using .PayoffFunctions
include("../utils/misc.jl")
using .MiscUtils
include("../particle_1D/plot_game_1D.jl")
using .PlotGame1DUtils
include("../particle_1D/plot_results_1D.jl")
using .PlotRes1D
include("../particle_1D/CPMP_bilin_game_1D.jl")
using .CPMPBilinGame1D
include("../particle_1D/game_theory_contdom_1D.jl")
using .GameTheoryContdom1DUtils
include("../particle_1D/lyapunov_1D.jl")
using .Lyapunov1DUtils

In [ ]:
ts = Dates.now()

alpha = 1
scaling = 1

## algo parameters
T = Int(floor(800 / (alpha^scaling)))
mx = 15 # nb particles
my = 15
eta0_wx = 4e-2 * alpha # initial stepsize
eta0_wy = 4e-2 * alpha
eta0_x  = 1e-3 * alpha
eta0_y  = 1e-3 * alpha
init_pos = "grid_unif" # "iid_unif" or "grid_unif"
extrasteps = 2 # extrasteps=1: CP-MDA, extrasteps=2: CP-MP

## random payoff matrix parameters
orderx = 3
ordery = 3

## plotting parameters
deltax = 1e-4 # discretization step when computing (approximation of) global NI error
deltay = 1e-4
evalNIevery    = Int(floor(T/50))
evallocNIevery = Int(floor(T/50))
evalVevery     = Int(floor(T/50))
plotiterevery  = Int(floor(T/50))

## logging
ts_fsfriendly = Dates.format(ts, "yyyy-mm-ddTHHMMSS") # filesystem-friendly string for ts
resultdir = mkpath("../results/particle__1Dorderx$(orderx)y$(ordery)__mx$(mx)y$(my)__extrasteps$(extrasteps)__$(ts_fsfriendly)")

logfile = "$resultdir/log.txt"
touch(logfile)
open(logfile, "a") do f
    write(f, "T=$T\n")
    write(f, "mx=$mx\n")
    write(f, "my=$my\n")
    write(f, "eta0_wx=$(eta0_wx)\n")
    write(f, "eta0_wy=$(eta0_wy)\n")
    write(f, "eta0_x=$(eta0_x)\n")
    write(f, "eta0_y=$(eta0_y)\n")
    write(f, "init_pos=$(init_pos)\n")
    write(f, "extrasteps=$(extrasteps)\n")
    write(f, "orderx=$(orderx)\n")
    write(f, "ordery=$(ordery)\n")
end

In [ ]:
## instantiate game
Random.seed!(rng, 1234)
gfun = random_fourier_function_1D(orderx, ordery; rng=rng)
heatmap_gfun_1D(gfun)

In [ ]:
## solve game
#==============#
copies_wx, copies_x, copies_wy, copies_y = run_CPMP(gfun, T, mx, my, eta0_wx, eta0_x, eta0_wy, eta0_y, init_pos; 
    extrasteps=extrasteps, 
    true_prox=true,
    rng=rng)
avg_wx, avg_x, avg_wy, avg_y = avg_iter(copies_wx, copies_x, copies_wy, copies_y)
save("$resultdir/iterates.jld", "copies_wx", copies_wx, "copies_x", copies_x, "copies_wy", copies_wy, "copies_y", copies_y) # save to JLD

In [ ]:
# Use the last iterate as reference point (estimate of the true MNE)
wx0, x0, wy0, y0 = copies_wx[:,T+1], copies_x[:,T+1], copies_wy[:,T+1], copies_y[:,T+1]
nierr0 = glob_NI_err(gfun, wx0, x0, wy0, y0)
gval0 = payoff(gfun, wx0, x0, wy0, y0)
open(logfile, "a") do f
    write(f, "Using last iterate as reference point (wx0, x0, wy0, y0) with NI error $(nierr0)\n")
    write(f, "Payoff at reference point: gval0=$(gval0)\n")
end
nierr0, gval0

In [ ]:
## compute and plot optimality metrics
#==============#
### compute and plot (global) NI error of iterates
thisT = Int(floor(T/2))
thisEvalNIevery = 2
plt_NI, _, nierrs = plot_NI_err(gfun, thisT, copies_wx, copies_x, copies_wy, copies_y, avg_wx, avg_x, avg_wy, avg_y,
    resultdir, logfile, thisEvalNIevery;
    deltax=deltax, deltay=deltay, skip_avg=true, hidetitle=true)
save("$resultdir/nierrs__every$(thisEvalNIevery)__t=1--$(thisT).jld", "nierrs", nierrs)
# nierrs = load("$resultdir/nierrs__every$(thisEvalNIevery)__t=1--$(thisT).jld", "nierrs")

In [ ]:
eps = 1e-10 # numerical stability (we use approximations (with deltax, deltay) to compute glob_NI_err)
upscale=2   # adjusting fontsize, linewidths etc.
resscale=2  # keep everything else fixed but increase resolution
fontsize=11/upscale*1.25

plt_NI_log = plot(range(1, stop=thisT+1, step=thisEvalNIevery), eps .+ max.(0, nierrs[1:thisEvalNIevery:thisT+1]), xlabel="k", yscale=:log10, label="",
    linewidth=upscale,
    xtickfontsize=fontsize, ytickfontsize=fontsize, xguidefontsize=fontsize, yguidefontsize=fontsize, legendfontsize=fontsize,
    yticks=[10^0, 10^(-3), 10^(-6)],
    dpi=upscale*100*resscale,
    size=(600/upscale, 400/upscale))
fn = "$resultdir/NI_errors_logscale_upscaled.png"
savefig(plt_NI_log, fn)
plt_NI_log

In [ ]:
### compute and plot local NI error of iterates
plt_locNI, plt_locNI_log, locnierrs = plot_locNI_err(gfun, T, copies_wx, copies_x, copies_wy, copies_y, avg_wx, avg_x, avg_wy, avg_y,
    resultdir, logfile, evallocNIevery)
plt_locNI_log

In [ ]:
## plot first variations at reference point
#==============#
len_xs=401
len_ys=401
xmin=0.
xmax=1.
ymin=0.
ymax=1.

upscale=2 # adjusting fontsize, linewidths etc.
resscale=2 # keep everything else fixed but increase resolution
fontsize=11/upscale*1.75

xs = range(xmin, stop=xmax, length=len_xs)
ys = range(ymin, stop=ymax, length=len_ys)
Fnu0_s = locmat(gfun, xs, y0) * wy0 # approximate continuous first-variation function w.r.t x
mu0F_s = transpose(wx0) * locmat(gfun, x0, ys) # approximate continuous first-variation function w.r.t y
mu0F_s = mu0F_s[:] # convert (*×1) Matrix to Vector

plt_firstvar_x = plot(xs, Fnu0_s, xlabel="x", label="",
    xticks=xmin:0.2:xmax,
    ylims=(-1.5, 1.5),
    linewidth=upscale,
    xtickfontsize=fontsize, ytickfontsize=fontsize, xguidefontsize=fontsize, yguidefontsize=fontsize, legendfontsize=fontsize,
    dpi=upscale*100*resscale,
    size=(600/upscale, 500/upscale))
hline!([gval0], linestyle=:dash, label="",
    linewidth=upscale)
vline!([x0], marker_z=wx0, color=:greens, colorbar_entry=false, label="",
    linewidth=upscale)

fn = "$resultdir/firstvar_x_withgreenlines.png"
savefig(plt_firstvar_x, fn)
plt_firstvar_x

In [ ]:
plt_firstvar_y = plot(ys, mu0F_s, xlabel="y", label="",
    xticks=ymin:0.2:ymax,
    ylims=(-3.25, -1.),
    # yticks=-4:0.5:-1,
    linewidth=upscale,
    xtickfontsize=fontsize, ytickfontsize=fontsize, xguidefontsize=fontsize, yguidefontsize=fontsize, legendfontsize=fontsize,
    dpi=upscale*100*resscale,
    size=(600/upscale, 500/upscale))
hline!([gval0], linestyle=:dash, label="", 
    linewidth=upscale)
vline!([y0], marker_z=wy0, color=:greens, colorbar_entry=false, label="",
    linewidth=upscale)

fn = "$resultdir/firstvar_y_withgreenlines.png"
savefig(plt_firstvar_y, fn)
plt_firstvar_y

In [ ]:
## plot iterations
#==============#
_, smoothsig_wx, smoothsig_wy = plot_iter_1D(gfun, wx0, x0, wy0, y0)

In [ ]:
len_xs=401
len_ys=401
xmin=0.
xmax=1.
ymin=0.
ymax=1.

upscale=2   # adjusting fontsize, linewidths etc.
resscale=2  # keep everything else fixed but increase resolution
fontsize=11/upscale*1.5

xs = range(xmin, stop=xmax, length=len_xs)
ys = range(ymin, stop=ymax, length=len_ys)

x01 = linrescale(x0, xmin, xmax)
y01 = linrescale(y0, ymin, ymax)
# smoothsig_wx = smoothen_iter(wx0, x01; len_s=len_xs)
# smoothsig_wy = smoothen_iter(wy0, y01; len_s=len_ys)

l = @layout [_ axx{0.7w,0.3h}; axy{0.3w,0.7h} axf]
axx = plot(xs, smoothsig_wx, 
    label="",
    xlabel="x",
    xlim=(xmin, xmax),
    xticks=xmin:0.2:xmax,
    yticks=false,
    ymirror=true, xmirror=true)
axy = plot(smoothsig_wy, ys, 
    label="",
    ylabel="y",
    ylim=(ymin, ymax),
    yticks=ymin:0.2:ymax,
    xticks=false,
    xflip=true, yflip=false)

axf = contour(xs, ys, gfun, fill=true, 
    # aspect_ratio=1, 
    colorbar_entry=false, showaxis=false)
vline!(x01, marker_z=wx0,
    # alpha=wx0,
    color=:greens, label="", colorbar_entry=false)
hline!(y01, marker_z=wy0,
    # alpha=wy0,
    color=:greens, label="", colorbar_entry=false)

pltt = plot(axx, axy, axf, layout=l,
    linewidth=upscale*0.75,
    xtickfontsize=fontsize, ytickfontsize=fontsize, xguidefontsize=fontsize, yguidefontsize=fontsize, legendfontsize=fontsize,
    dpi=upscale*100*resscale,
    size=(600/upscale, 600/upscale))

fn = "$resultdir/last_iter.png"
savefig(pltt, fn)
pltt

In [ ]:
# plot_iters(gfun, T, copies_wx, copies_x, copies_wy, copies_y, avg_wx, avg_x, avg_wy, avg_y,
#     resultdir,
#     plotiterevery)

In [ ]:
## plot lyapunov function
#==============#
### Get aggregated representation of the reference point (ideally, should check manually!)
wxstar, xstar, mxstar = aggregate_particles_1D(wx0, x0; thresh_pos=1e-5)
wystar, ystar, mystar = aggregate_particles_1D(wy0, y0; thresh_pos=1e-5)
dxstar_min = minimum([torus_dist_1D(xstar[I], xstar[J]) for I=1:mxstar for J=1:I-1])
dystar_min = minimum([torus_dist_1D(ystar[I], ystar[J]) for I=1:mystar for J=1:I-1])
open(logfile, "a") do f
    write(f, "mxstar, mystar, dxstar_min, dystar_min: $mxstar, $mystar, $dxstar_min, $dystar_min\n")
end
mxstar, mystar, dxstar_min, dystar_min

In [ ]:
alpha_x = 3
alpha_y = 3
lambda_x = 3 # (must tweak manually! paper uses lambda_x = eta_x^{-1/6} for proofs)
lambda_y = 3
tau_x = 1e-2
tau_y = 1e-2

In [ ]:
thisT = Int(floor(T/2))
thisEvalVevery = 1

# Cleaner plot to put in paper
plt_Vxy, _, Vweis_xy, Vposs_xy, Vtots_xy = plot_V_xy(gfun, thisT, copies_wx, copies_x, copies_wy, copies_y, avg_wx, avg_x, avg_wy, avg_y,
    wxstar, xstar, wystar, ystar, 
    eta0_wx, eta0_x, eta0_wy, eta0_y,
    resultdir, logfile,
    thisEvalVevery;
    hidetitle=true,
    alpha_x=alpha_x, lambda_x=lambda_x, tau_x=tau_x,
    alpha_y=alpha_y, lambda_y=lambda_y, tau_y=tau_y)

In [ ]:
eps = 1e-30 # numerical stability
upscale=2   # adjusting fontsize, linewidths etc.
resscale=2  # keep everything else fixed but increase resolution
fontsize=11/upscale*1.25

plt_Vxy_log = plot(range(1, stop=thisT, step=thisEvalVevery), eps.+ max.(0, Vtots_xy[1:thisEvalVevery:thisT]), xlabel="k", label="total", linewidth=upscale, linealpha=0.75, yscale=:log10,
    xtickfontsize=fontsize, ytickfontsize=fontsize, xguidefontsize=fontsize, yguidefontsize=fontsize, legendfontsize=fontsize,
    dpi=upscale*100*resscale,
    size=(600/upscale, 400/upscale))
plot!(range(1, stop=thisT, step=thisEvalVevery), eps.+ max.(0, Vweis_xy[1:thisEvalVevery:thisT]), xlabel="k", line=:dot, label="weight term", linewidth=upscale*0.75, yscale=:log10)
plot!(range(1, stop=thisT, step=thisEvalVevery), eps.+ max.(0, Vposs_xy[1:thisEvalVevery:thisT]), xlabel="k", line=:dash, label="position term", linewidth=upscale*0.5, yscale=:log10)
fn = "$resultdir/Vxy_logscale.png"
savefig(plt_Vxy_log, fn)
plt_Vxy_log